In [1]:
import pymarc
import gzip
import random
import json
from bookwormMARC.bookwormMARC import parse_record

In [2]:
import os
os.getcwd()

'/Volumes/drobo/hathi_metadata'

The following code block from https://github.com/rlmv/hathitrust-soer/blob/master/marc/records.py

It works on publication dates. Does it also work on author birth and death years?

In [3]:
%load_ext autoreload
%autoreload 2

In [123]:
# This would be easier to demo if this could be loaded over the web. But that apparently will take Python 3.
#import urllib2

#file = urllib2.urlopen("https://dpla-provider-export.s3.amazonaws.com/2016/04/hathitrust.json.gz")

hathi_records = gzip.open("json/hathitrust.json.gz")

Here is an example of the output of this script on Hathi books: 5 randomly selected records from the first 50000 or so in the DPLA dump. This is usually more than 5 *items*: Hathi groups multiple items into a single record.

In [135]:
n=0
for line in hathi_records:
    if random.random() > .0001:
        # Print random files each time for debugging
        continue
    n+=1
    if line.startswith(","):
        # This lets us parse the JSON array 1 row at a time.
        line = line[1:]
    if line == "[\n" or line=="]\n":
        # They helpfully format the JSON so the array bars are on their own lines
        continue
    rec = parse_record(line)
    for entry in rec.hathi_bookworm_dicts():
        print json.dumps(entry)
        print ""
    if n>5:
        break


{"libraryB": "HVD", "lc_class_from_lc": true, "searchstring": "<a href=https://babel.hathitrust.org/cgi/pt?id=hvd.hn6gke><em>The life of the Rev. John Owen, D.D., abridged from Orme's Life of Owen.</em> (1840)", "lc2": "9225.", "scanner": "google", "lc0": "B", "lc1": "BX", "first_author_death": 1830, "title": "The life of the Rev. John Owen, D.D., abridged from Orme's Life of Owen.", "permalink": "https://babel.hathitrust.org/cgi/pt?id=hvd.hn6gke", "filename": "hvd.hn6gke", "libraryC": "HVD", "date": 1840, "first_author_birth": 1787, "first_place": "Philadelphia,", "first_author_name": "Orme, William, 1787-1830.", "first_publisher": "Presbyterian board of publication,", "date_source": "008"}

{"libraryB": "HVD", "permalink": "https://babel.hathitrust.org/cgi/pt?id=hvd.32044068333434", "searchstring": "<a href=https://babel.hathitrust.org/cgi/pt?id=hvd.32044068333434><em>Shuo jiao /</em> (1896)", "scanner": "google", "title": "Shuo jiao /", "filename": "hvd.32044068333434", "libraryC": 

The following block shows that something like 1 in 3 items, in about one in ten records, have a different entry in the 974y field from the native date field. That suggests huge possibilities for improving dates if we're not already using the 974y fields: I suspect we are not based on the serial volumes that include 974y fields I see in the online browser.

In [134]:
records = 0
diff_records = 0
items = 0
diff_items = 0
for line in hathi_records:
    if random.random() > .01:
        # Print random files each time for debugging
        continue
    records += 1
    if line.startswith(","):
        # This lets us parse the JSON array 1 row at a time.
        line = line[1:]
    if line == "[\n" or line=="]\n":
        # They helpfully format the JSON so the array bars are on their own lines
        continue
    rec = parse_record(line)
    line_counted = False
    for field in rec.get_fields('974'):
        items += 1
        if str(field['y']) != str(rec.date()):
            if not line_counted:
                diff_records += 1
                line_counted = True
            diff_items +=1
    if records>1000:
        break

        
        
print "%i out of %i records and %i out of %i items have differing dates" %(diff_records,records,diff_items,items)

147 out of 1001 records and 660 out of 1854 items have differing dates


In [122]:
records

0